<a href="https://colab.research.google.com/gist/yookontian/7d9a11addf2b9b90c8ca91ca28af8264/haiku-kanji-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PIP Things

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.2 MB/s 
     |████████████████████████████████| 72 kB 1.6 MB/s 
     |████████████████████████████████| 5.5 MB 63.2 MB/s 
     |████████████████████████████████| 212 kB 64.0 MB/s 
     |████████████████████████████████| 163 kB 64.2 MB/s 
     |████████████████████████████████| 115 kB 70.8 MB/s 
     |████████████████████████████████| 95 kB 5.5 MB/s 
     |████████████████████████████████| 127 kB 54.5 MB/s 
     |████████████████████████████████| 115 kB 59.8 MB/s 
     |████████████████████████████████| 7.6 MB 43.6 MB/s 
     |████████████████████████████████| 1.3 MB 42.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      S

In [ ]:
!pip install mecab-python3 logzero sentencepiece fugashi ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 5.1 MB/s 
     |████████████████████████████████| 583 kB 68.3 MB/s 
     |████████████████████████████████| 13.4 MB 62.1 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556722 sha256=797266105c9ff28fc197ae0c383983cc41ce8d09493fc236ca8b57c95b792feb
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [ ]:
!pip install unidic-lite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47.4 MB 116 kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658837 sha256=b5449ffd609d85e4ff9498d1f368e001e63d58884f35219cabf09d2d279e97ce
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite


### Introduce the pre-trained model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
checkpoint = "cl-tohoku/bert-base-japanese-char-v2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForMaskedLM.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/516 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/366M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-char-v2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='cl-tohoku/bert-base-japanese-char-v2', vocab_size=6144, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
sequence = "ラジオを消すゼラチン質の闇の音"
tokens = tokenizer.tokenize(sequence)

In [ ]:
tokens

['ラ', 'ジ', 'オ', 'を', '消', 'す', 'ゼ', 'ラ', 'チ', 'ン', '質', 'の', '闇', 'の', '音']

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
ids

[1011,
 962,
 948,
 932,
 3247,
 875,
 966,
 1011,
 971,
 1021,
 5061,
 896,
 5505,
 896,
 5657]

In [ ]:
tokenizer.convert_ids_to_tokens(ids)

['ラ', 'ジ', 'オ', 'を', '消', 'す', 'ゼ', 'ラ', 'チ', 'ン', '質', 'の', '闇', 'の', '音']

### Loading data

In [ ]:
from datasets import load_dataset
dataset = load_dataset("text", data_dir="1/")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-7723450e588b7549/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset.keys()

dict_keys(['train'])

In [ ]:
def tokenize_function(example):
    return tokenizer(example['text'],add_special_tokens=True, truncation=True,
                     padding=True, return_tensors='pt')

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/90 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 89821
    })
})

In [ ]:
## detele the 'text' column

# tokenized_datasets = tokenized_datasets.remove_columns(['text'])

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)

### Training model

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(checkpoint)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-char-v2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text. If text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 89821
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 33684
  Number of trainable parameters = 90767616


Step,Training Loss
500,3.638800
1000,3.519200
1500,3.447900
2000,3.326000
2500,3.369900
3000,3.303000
3500,3.315400
4000,3.275200
4500,3.259900
5000,3.186100


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1500
Configuration saved in test-trainer/checkpoint-1500/config.json
Model weights saved in test-trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved

TrainOutput(global_step=33684, training_loss=2.8270363675742516, metrics={'train_runtime': 3287.3509, 'train_samples_per_second': 81.97, 'train_steps_per_second': 10.247, 'total_flos': 4507664844925440.0, 'train_loss': 2.8270363675742516, 'epoch': 3.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to test-trainer
Configuration saved in test-trainer/config.json
Model weights saved in test-trainer/pytorch_model.bin
tokenizer config file saved in test-trainer/tokenizer_config.json
Special tokens file saved in test-trainer/special_tokens_map.json
